In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [12]:
data = yf.download("Gold", start="2018-01-01", end="2022-01-01")
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,14.65,15.24,14.61,15.20,13.524041,15223200
2018-01-03,15.24,15.27,14.74,15.00,13.346095,15155900
2018-01-04,15.03,15.05,14.79,15.03,13.372786,8936100
2018-01-05,14.99,15.00,14.87,14.95,13.301607,6515600
2018-01-08,14.88,14.95,14.64,14.74,13.114763,8362600


In [13]:
data.drop(columns=["Adj Close"])

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-02,14.650000,15.240000,14.610000,15.200000,15223200
2018-01-03,15.240000,15.270000,14.740000,15.000000,15155900
2018-01-04,15.030000,15.050000,14.790000,15.030000,8936100
2018-01-05,14.990000,15.000000,14.870000,14.950000,6515600
2018-01-08,14.880000,14.950000,14.640000,14.740000,8362600
...,...,...,...,...,...
2021-12-27,18.500000,18.580000,18.270000,18.410000,11740600
2021-12-28,18.450001,18.670000,18.309999,18.370001,10310100
2021-12-29,18.240000,18.660000,18.219999,18.400000,17290100


In [14]:
target = data["High"]
target = target.to_numpy()
data.drop(columns="High", inplace=True)

In [15]:
scaler_data = StandardScaler()
scaler_target = StandardScaler()

data = scaler_data.fit_transform(data)
data = pd.DataFrame(data, columns=["date", "open", "low", "close", "volume"])

target = scaler_target.fit_transform(target.reshape(-1, 1))
target = pd.DataFrame(target, columns=["max"])

data.drop(columns=["date"], inplace=True)

In [17]:
with open('scaler_data.pkl', 'wb') as f:
    pickle.dump(scaler_data, f)

with open('scaler_target.pkl', 'wb') as f:
    pickle.dump(scaler_target, f)

In [33]:
pre_data = [-1:]
with open('data.pkl', 'wb') as f:
    pickle.dump(scaler_data, f)

[[[ 0.03297055  0.01961309  0.17411571 -0.3265938 ]
  [ 0.02910916  0.04617586  0.20105831 -0.15259094]
  [ 0.07158334  0.05376497  0.20875647 -0.58005578]
  [ 0.04841574  0.02340782  0.17796459 -0.64149758]
  [ 0.05613815  0.01581871  0.17026643 -0.84918309]
  [ 0.03876227  0.02151045  0.17603977  0.16420024]
  [ 0.07158334  0.10119806  0.25686911 -0.37576757]]]


In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [20]:
model = Sequential([
    LSTM(64, input_shape=(lookback, x_train.shape[2])),
    Dense(64, activation="relu"),
    Dense(1)
])

2023-04-19 08:22:55.367706: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-19 08:22:55.666727: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 08:22:55.670933: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-19 08:22:55.674718: I tensorflow/core/com

In [21]:
filepath="model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [22]:
model.compile(optimizer='adam', loss='mse', metrics=["mse"])

In [23]:
#history = model.fit(x_train, y_train, epochs=64, batch_size=16, validation_data=(x_test, y_test), callbacks=[checkpoint])

In [24]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()

NameError: name 'history' is not defined

In [ ]:
model.evaluate(x_test, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 0.0075 - mse: 0.0075


[0.0074523077346384525, 0.0074523077346384525]

In [ ]:
result = model.predict(x_test[85].reshape(1, x_test[0].shape[0], x_test[0].shape[1]))

1/1 [==============================] - 0s 396ms/step


2023-04-19 01:23:33.912826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 01:23:33.914947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-19 01:23:33.916417: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
with open('scaler_target.pkl', 'rb') as f:
    scaler_target = pickle.load(f)

In [ ]:
result = scaler_target.inverse_transform(result)
print(result)

[[19.803875]]


In [ ]:
answer = scaler_target.inverse_transform(y_test[85].reshape(1, y_test[0].shape[0]))
print(answer)

[[17.97999954]]


In [ ]:
from keras.models import load_model

model = load_model('model.h5')

2023-04-19 01:24:13.587651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 01:24:13.589894: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-19 01:24:13.591475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus